# Robot functions (to be completed)

In [1]:
# !pip install math3d

In [2]:
import json
import urx
import robotiq_gripper

# ip1 = "192.168.2.207"   #ip of the UR robot to connect

# ip2 = ""   #???????

# gripper_1 = robotiq_gripper.RobotiqGripper()  # initialize the gripper
# gripper_2 = robotiq_gripper.RobotiqGripper()   #???????
# print("Connecting to grippers...")


# gripper_1.connect(ip1, 63352)                  # connect to the gripper
                                           
# robot_1 = urx.Robot(ip1, use_rt=True)            # connect to the UR robot


# gripper_2.connect(ip2, 63352)                  # connect to the gripper
                                           
# robot_2 = urx.Robot(ip2, use_rt=True)            # connect to the UR robot



# def move_top(robot_id, gripper_id , object_position):  
#     return ""

# def gripper_open(robot_id, gripper_id):
#     gripper_id.move_and_wait_for_pos(0,100,100)



############################# functions used in prompting  

def move_to_object(robot_id, gripper_id, object_position): # to move to the position of an object
    print(" move_to_object(robot_id, gripper_id, object_position) ")
    return " move_to_object(robot_id, gripper_id, object_position) " # for testing


def grasp_object(robot_id, gripper_id, object_position):  # to grasp an object
    print(" grasp_object(robot_id, gripper_id, object_position)" )
    return " grasp_object(robot_id, gripper_id, object_position) " # for testing

def connect(robot_id, gripper_id, object_1_position, object_2_position): # connect a part to another
    print(" connect(robot_id, gripper_id, object_1_position, object_2_position)")
    return " connect(robot_id, gripper_id, object_1_position, object_2_position)" # for testing

def hold_object(robot_id, gripper_id, object_position): # To apply a forcce on the base to avoid it moving
    print(" hold_object(robot_id, gripper_id, object_position) " )
    return " hold_object(robot_id, gripper_id, object_position) " # for testing



#############################


def move_to_object_wrapper(state):  #inputs = {"messages": [steps_list[0],'Robot1_base_holder', 'gripper1', "{'x': -25", "'y': 0", "'alpha': 0}"]}
    return  move_to_object()


In [3]:
# ! pip install langgraph
# ! pip install langchain_openai

## JSON file details extraction 

In [4]:
import sys
import json

class AssemblyEnvironment:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            data = json.load(file)
        
        self.base = data['assembly']['base']
        self.base_num_pins=len(self.base["pins"])
        self.parts = data['assembly']['parts']
        self.num_parts = len(self.parts)
        self.base_position = data['assembly']['base']['position']

        # Dynamically create pin attributes
        for i in range(len(self.base["pins"])):
            pin_name = f"base_pin_{i + 1}"  # Create dynamic pin names
            setattr(self, pin_name, self.base["pins"][i])
        
        # Extract part details dynamically
        for i, part in enumerate(self.parts):
            part_id = part["id"]
            setattr(self, f"{part_id}_position", part["position"])
            setattr(self, f"{part_id}_target_position", part["target_position"])
            setattr(self, f"{part_id}_mounting_hole", part["mounting_hole"])
            setattr(self, f"{part_id}_grip_pin", part["grip_pin"])

        self.assembly_prompt  = lambda user_query: f""" 
You are an AI agent which task is to provide a detail plan for fully assembling an object of {self.num_parts + 1 } different components. 
The object has a base with { self.base_num_pins } pins where the  {self.num_parts} others parts which all contains amounting hole have to be mounted.
The object have to be assembled by two robot (Robot1_base_holder ,Robot2_assembler) manipulator each equipped with a gripper (gripper1, gripper2)and 
your goal is to provide a deatailed plan on how the to robots have to collaborate togother to solve the assembly task by strictly using 
the following functions to control the motions of the two robots: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position).

In order to answer use the following template:

START OF PLAN
 step 1 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should move to the position of the base using the function [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
   grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position)]
 step 2 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should hold the base using the function [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 step 3 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  shoul move to the positon of [CHOICE: part_1, part_2, ....] with the function  [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), connect(robot_id, gripper_id, object_1_position, object_2_position)]
 step 4 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should move grasp [CHOICE: part_1, part_2, ....] with the function  [CHOICE: move_to_object(robot_id, gripper_id, object_position), 
grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 step 5 - [CHOICE:  Robot1_base_holder,Robot2_assembler]  should connect  [CHOICE: part_1, part_2, ....] to the pin  [CHOICE: pin_1, pin_2, ....] on the base with the function
[CHOICE: move_to_object(robot_id, gripper_id, object_position), grasp_object(robot_id, gripper_id, object_position),  hold_object(robot_id, gripper_id, object_position), 
connect(robot_id, gripper_id, object_1_position, object_2_position)]
 step 6 - .....
END OF PLAN


Rules:
1. your answer should explicitly start with 'START OF PLAN' and end it with 'END OF PLAN' .
2. When you see phrases like [CHOICE: choice1, choice2, ...], you should replace the entire phrase with only one of the choices listed.
3. only Robot1_base_holder should hold the base for satability reasons when Robot2_assembler is doing the assembly task


Example output:
START OF PLAN
 1 - Robot1_base_holder should move to the position of the base using the function move_to_object(robot1_base_holder, gripper1, base_position), 
 2  .....
END OF PLAN


Now here is the user given task : {user_query}
"""    

if __name__ == "__main__":
    # Redirect stdout to a file
    original_stdout = sys.stdout  # Save a reference to the original standard output

    with open('output.txt', 'w') as f:
        sys.stdout = f  # Change the standard output to the file we created.

        # Create an instance of AssemblyEnvironment by providing the path to the JSON file
        assembly_env = AssemblyEnvironment('assembly_details.JSON')

        
        print("On The Base:")
        
        # Print the base and pins to verify
        for pin in assembly_env.base["pins"]:
            pin_id = pin["id"]
            pin_position = pin["position"]
            print(f"Pin {pin_id} Position on the base:", pin_position)  # Print pin position


        print("\n")
        print(f"The {assembly_env.num_parts} assembly parts:")
        
        # Accessing dynamic part attributes with detailed descriptions
        for part in assembly_env.parts:
            part_id = part["id"]
            print(f"Part {part_id}:")
            # print(f"{part_id} Position:", getattr(assembly_env, f"{part_id}_position"))  # Position of the part
            print(f"{part_id} Target Position:", getattr(assembly_env, f"{part_id}_target_position"))  # Target position of the part
            print(f"{part_id} Mounting Hole:", getattr(assembly_env, f"{part_id}_mounting_hole"))  # Mounting hole details of the part
            print(f"{part_id} Grip Pin:", getattr(assembly_env, f"{part_id}_grip_pin"))  # Grip pin position of the part
            print("-----------")
            
        sys.stdout = original_stdout  # Reset the standard output to its original value

print("All output has been saved to output.txt")
# print(assembly_env.assembly_prompt("hollllllllllllllllllllllllllla"))

All output has been saved to output.txt


## RAG System

In [5]:
# !pip install langchain-community
# !pip install sentence_transformers
# !pip install --upgrade chromadb


In [6]:
################################################################ RAG



from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

### Reading the txt files from source directory

loader = DirectoryLoader('./', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
# retriever = db.as_retriever(search_kwargs={"k": 4})
retriever = db.as_retriever(search_kwargs={"k": 1})




####################### testing RAG
query = "what is the Mounting Hole of part 2"
docs = retriever.invoke(query)
print(docs)

/home/koffi/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


[Document(metadata={'source': 'output.txt'}, page_content="part_2 Mounting Hole: {'id': 2, 'position': {'x': 0, 'y': 0, 'alpha': 0}}")]


## Building agents

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()


from langchain_openai import ChatOpenAI

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI


import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

from langchain_core.tools import BaseTool

from langchain_core.tools import BaseTool



from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA


import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langgraph.graph import Graph

In [8]:


# Now you can access your environment variables using os.environ
os.environ['OPENAI_API_KEY'] = ''
# os.environ['TAVILY_API_KEY'] = os.environ.get("TAVILY_API_KEY")


llm = ChatOpenAI(model="gpt-3.5-turbo")
# llm = ChatOpenAI(model="gpt-4-turbo")

## Graph 1

In [9]:
def transform_plan(plan_text):
    # Split the input text into lines
    lines = plan_text.strip().split('\n')
    
    # Initialize an empty list to hold the formatted output
    formatted_lines = []
    
    # Flag to indicate if we are within the plan section
    in_plan = False
    
    # Iterate over the lines
    for line in lines:
        line = line.strip()
        if line == "START OF PLAN":
            in_plan = True
            continue
        elif line == "END OF PLAN":
            break
        
        # If we are in the plan section, format the line
        if in_plan and line:
            formatted_lines.append(f'"{line}"')
    
    # Return the formatted lines joined by newline characters
    return '\n'.join(formatted_lines)

In [37]:

def generate_plan_step1(state):
    print('-> ...Plan Generation 1... ->')
    messages = state['messages']
    user_query = messages[-1]   ## Fetching the user question
    complete_query = assembly_env.assembly_prompt( user_query) # to do
    response = llm.invoke(complete_query)
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    print('-> ...Plan Generation 1 Done... ->')
    return state



def generate_plan_step2(state, filename = 'output.txt' ):
    print('-> ...Plan Generation 2... ->')
    
    messages = state['messages']
    plan = messages[-1] ## Fetching the user question

    with open(filename, 'r') as file:
        context = file.read() #to check later 
    
    question = "Now here is the plan to fill up: \n " + plan
    # print('Question:',question)

    template_format = lambda ctext, quest : f"""You will recieve a plan generated by an AI agent which contain a list of step that two robot manipulators 
have to follow in order to perform a user given task. The plan contain some functions and your goal is to replace the arguments 
of the function by their real values. All the reals variables values can be find in the following retrieved context:\n
{ctext}

Example: 
If the plan is like:
step 1 - Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, base_position)
then your response should look like:
step 1 - Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, [-25, 0, 0])

Question: {quest}
    """
    template = template_format(context,question)
    print('template:',template)
    # prompt = ChatPromptTemplate.from_template(template)
    # print("debug")
    # retrieval_chain = (
    #     {"context": retriever, "question": RunnablePassthrough()}
    #     | prompt
    #     | llm
    #     | StrOutputParser()
    #     )
    # result = retrieval_chain.invoke(question)

    response = llm.invoke(template)
    # clean_response = transform_plan(response.content)
    # # clean_response = response
    state["messages"] = []
    # state['messages'].append(clean_response) # appending LLM call response to the AgentState
    state['messages'].append(response.content) # appending LLM call response to the AgentState
    print('-> ...Plan Generation 2 Done... ->')
    return state
    
    # return result
    


In [11]:
# Define a Langchain graph

# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []


workflow1 = Graph()

# workflow.add_node("Agent", function_1)
# workflow.add_node("tool", function_2)
# workflow1 = Graph().add_edge('Agent', 'tool')
workflow1.add_node("generate_plan1", generate_plan_step1)
workflow1.add_node("generate_plan2", generate_plan_step2)
workflow1.add_edge('generate_plan1', 'generate_plan2')

workflow1.set_entry_point("generate_plan1")
workflow1.set_finish_point("generate_plan2")

app1 = workflow1.compile()

In [12]:
inputs = {"messages": ["assemble a gripper of one base and two parts for me"]}
out =  app1.invoke(inputs)

-> ...Plan Generation 1... ->
-> ...Plan Generation 1 Done... ->
-> ...Plan Generation 2... ->
template: You will recieve a plan generated by an AI agent which contain a list of step that two robot manipulators 
have to follow in order to perform a user given task. The plan contain some functions and your goal is to replace the arguments 
of the function by their real values. All the reals variables values can be find in the following retrieved context:

On The Base:
Pin 1 Position on the base: {'x': -25, 'y': 0, 'alpha': 0}
Pin 2 Position on the base: {'x': 25, 'y': 0, 'alpha': 0}


The 2 assembly parts:
Part part_1:
part_1 Target Position: {'x': -25, 'y': 0, 'alpha': 0}
part_1 Mounting Hole: {'id': 1, 'position': {'x': 0, 'y': 0, 'alpha': 0}}
part_1 Grip Pin: {'position': {'x': -40, 'y': 15, 'alpha': 0}}
-----------
Part part_2:
part_2 Target Position: {'x': 25, 'y': 0, 'alpha': 0}
part_2 Mounting Hole: {'id': 2, 'position': {'x': 0, 'y': 0, 'alpha': 0}}
part_2 Grip Pin: {'position':

In [13]:
out['messages']

["START OF PLAN\n step 1 - Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})\n step 2 - Robot1_base_holder should hold the base using the function grasp_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})\n step 3 - Robot2_assembler should move to the position of part_1 using the function move_to_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})\n step 4 - Robot2_assembler should grasp part_1 using the function grasp_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})\n step 5 - Robot2_assembler should connect part_1 to pin_1 on the base using the function connect(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0}, {'x': -25, 'y': 0, 'alpha': 0})\n step 6 - Robot2_assembler should move to the position of part_2 using the function move_to_object(Robot2_assembler, gripper2, {'x': 25, 'y': 0, 'alpha': 0})\n step 7 - Robot2_assemb

In [17]:
len(out['messages'])

1

## Graph 2

In [18]:
def extract_steps(plan_text):
    # Split the input text into lines
    lines = plan_text.splitlines()
    
    # Initialize an empty list to hold the steps
    steps = []
    
    # Flag to check if we are within the plan
    in_plan = False
    
    for line in lines:
        # Check for the start and end of the plan
        if "START OF PLAN" in line:
            in_plan = True
            continue
        elif "END OF PLAN" in line:
            break
        
        # If we are within the plan, extract the step
        if in_plan and line.strip():
            # Extract the step description
            step = line.split(" - ", 1)[1].strip() if " - " in line else line.strip()
            steps.append(step)
    
    return steps


# Extracting steps
steps_list = extract_steps(out['messages'][0]) 
for i in steps_list:
    print(i)

Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})
Robot1_base_holder should hold the base using the function grasp_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})
Robot2_assembler should move to the position of part_1 using the function move_to_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})
Robot2_assembler should grasp part_1 using the function grasp_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})
Robot2_assembler should connect part_1 to pin_1 on the base using the function connect(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0}, {'x': -25, 'y': 0, 'alpha': 0})
Robot2_assembler should move to the position of part_2 using the function move_to_object(Robot2_assembler, gripper2, {'x': 25, 'y': 0, 'alpha': 0})
Robot2_assembler should grasp part_2 using the function grasp_object(Robot2_assembler, gripper2, {'x': 25,

In [19]:
steps_list

["Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})",
 "Robot1_base_holder should hold the base using the function grasp_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})",
 "Robot2_assembler should move to the position of part_1 using the function move_to_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})",
 "Robot2_assembler should grasp part_1 using the function grasp_object(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0})",
 "Robot2_assembler should connect part_1 to pin_1 on the base using the function connect(Robot2_assembler, gripper2, {'x': -25, 'y': 0, 'alpha': 0}, {'x': -25, 'y': 0, 'alpha': 0})",
 "Robot2_assembler should move to the position of part_2 using the function move_to_object(Robot2_assembler, gripper2, {'x': 25, 'y': 0, 'alpha': 0})",
 "Robot2_assembler should grasp part_2 using the function grasp_object(Robot2_asse

In [31]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate


class TopicSelectionParser(BaseModel):
    Topic: str = Field(description='Selected Topic')
    #Reasoning: str = Field(description='Reasoning behind topic selection')

parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)


from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [35]:
# input type: 1. **step 1** - Robot1_base_holder should move to the position of the base using the 
# function move_to_object(robot1_base_holder, gripper1, {\'x\': 0, \'y\': 0, \'alpha\': 0})

def clf_functions(state):
    print('-> Calling classifier ->')
    messages = state['messages']
    # question = messages[-1]   ## Fetching the user question
    question = messages[0]
    
    templete = """ Your task is choose the right function that have to be used based on the input query. 
        Only output the function among: [move_to_object ,grasp_object , connect, hold_object ]. 
        Don't include reasoning. 
        Following is the input query:  {question}
        {format_instructions} """
    prompt = PromptTemplate(template=templete,
                                    input_variables=[question],
                                    partial_variables={
                                        "format_instructions" : parser.get_format_instructions()                                    }
                                    )
    chain = prompt | llm | parser

    response = chain.invoke({"question":question,"format_instructions" : parser.get_format_instructions() })

    print(response)

    state["messages"][0] = response.Topic

    # return {"messages": [response.Topic]}
    return state


def router(state):
    print('-> Router ->')
    
    messages = state["messages"]
    last_message = messages[0]
    print(last_message)
    if 'move_to_object' in last_message:
        return 'move_to_object Call'
    elif 'grasp_object' in last_message:
        return 'grasp_object Call'
    elif 'connect' in last_message:
        return 'connect Call'
    elif 'hold_object' in last_message:
        return 'hold_object Call'
    else:
        return 'Sorry Sorry Sorry'





from langgraph.graph import StateGraph,END

graph = StateGraph(AgentState) ### StateGraph with AgentState

graph.add_node("move_to_object", move_to_object)
graph.add_node("grasp_object", grasp_object)
graph.add_node("connect", connect)
graph.add_node("hold_object", hold_object)
graph.add_node("clf functions", clf_functions)

graph.set_entry_point("clf functions")


# conditional edges are controlled by our router
graph.add_conditional_edges(
    "clf functions",
    router,
    {
        "move_to_object Call": "move_to_object",
        "grasp_object Call": "grasp_object",
        "connect Call":"connect",
        "hold_object Cal":"hold_object",
    }
)


graph.add_edge("move_to_object", END)
graph.add_edge("connect", END)
graph.add_edge("grasp_object", END)
graph.add_edge("hold_object", END)

app2 = graph.compile()


In [36]:
inputs = {"messages": [steps_list[0],'Robot1_base_holder', 'gripper1', "{'x': -25", "'y': 0", "'alpha': 0}"]}
out = app2.invoke(inputs)

-> Calling classifier ->
Topic='move_to_object'
-> Router ->
move_to_object


TypeError: move_to_object() missing 2 required positional arguments: 'gripper_id' and 'object_position'

{'messages': ["Robot1_base_holder should move to the position of the base using the function move_to_object(Robot1_base_holder, gripper1, {'x': -25, 'y': 0, 'alpha': 0})"]}

In [ ]:

# def router(state):
#     print('-> Router ->')
    
#     messages = state["messages"]
#     last_message = messages[-1]
#     print(last_message)
#     if 'Japan' in last_message or 'Sports' in last_message:
#         return 'RAG Call'
#     else:
#         return 'LLM Call'

# from langgraph.graph import StateGraph,END

# graph = StateGraph(AgentState) ### StateGraph with AgentState

# graph.add_node("agent", function_1)
# graph.add_node("RAG", function_2)
# graph.add_node("LLM", function_3)

# graph.set_entry_point("agent")


# # conditional edges are controlled by our router
# graph.add_conditional_edges(
#     start_key="agent",  # where in graph to start
#     condition=router,  # function to determine which node is called
#     conditional_edge_mapping={
#         "RAG Call": "RAG",
#         "LLM Call": "LLM",
#     }
# )

# graph.add_edge("RAG", END)
# graph.add_edge("LLM", END)

# app = graph.compile()

In [ ]:
# inputs = {"messages": ["Tell me about Japan's Industrial Growth"]}
# out = app.invoke(inputs)

In [ ]:
# def determine_call_type(last_message):
#     if 'move_to_object' in last_message:
#         return 'RAG Call'
#     elif 'grasp_object' in last_message:
#         return 'RAG Call'
#     elif 'connect' in last_message:
#         return 'RAG Call'
#     elif 'hold_object' in last_message:
#         return 'RAG Call'
#     else:
#         return 'LLM Call'

In [ ]:
# def move_to_object(robot_id, gripper_id, object_position): # to move to the position of an object
#     return " move_to_object(robot_id, gripper_id, object_position) " # for testing


# def grasp_object(robot_id, gripper_id, object_position):  # to grasp an object
#     return " grasp_object(robot_id, gripper_id, object_position) " # for testing

# def connect(robot_id, gripper_id, object_1_position, object_2_position): # connect a part to another
#     return " connect(robot_id, gripper_id, object_1_position, object_2_position)" # for testing

# def hold_object(robot_id, gripper_id, object_position): # To apply a forcce on the base to avoid it moving
#     return " hold_object(robot_id, gripper_id, object_position) " # for testing